In [1]:
import mlflow

In [4]:
from mlflow import MlflowClient

In [6]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

In [7]:
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='/workspaces/MLOps/01-intro/mlruns/3', creation_time=1728050588669, experiment_id='3', last_update_time=1728050588669, lifecycle_stage='active', name='NYC-Taxi-Regressor', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1727863982950, experiment_id='0', last_update_time=1727863982950, lifecycle_stage='active', name='Default', tags={}>]


In [23]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids= "3",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=3,
    order_by=["metrics.rmse ASC"]
)

In [24]:
runs[0].data

<RunData: metrics={'rmse': 18.33542862192436}, params={'bootstrap_type': 'Bernoulli',
 'depth': '8',
 'early_stopping_rounds': '100',
 'eval_metric': 'RMSE',
 'iterations': '631',
 'l2_leaf_reg': '7',
 'learning_rate': '0.2194813656713647',
 'loss_function': 'RMSE'}, tags={'mlflow.log-model.history': '[{"run_id": "1e40c7e0866c4012901bbc81fd564e8c", '
                             '"artifact_path": "models_mlflow", '
                             '"utc_time_created": "2024-10-04 '
                             '16:17:23.388746", "model_uuid": '
                             '"48b0942132564a18bd429b670ede63b0", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.catboost", "python_version": "3.12.4", '
                             '"data": "model.cb", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"catboost": {"catboost_version": "1.2.7", '
 

In [26]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']}")

run id: 1e40c7e0866c4012901bbc81fd564e8c, rmse: 18.33542862192436
run id: 4238514fde034ece81b6855a21e122e2, rmse: 18.33542862192436
run id: 2cd45b7cc3b7475ca3c0b4d279a3af66, rmse: 18.403313330771905


In [27]:
RUN_ID = "1e40c7e0866c4012901bbc81fd564e8c"
MODEL_URI = f"runs:/{RUN_ID}/model"

## But first let's ensure that the run has not been registered:


In [28]:
assert client.search_model_versions(f"run_id = '{RUN_ID}'") == [], "Run has been registered!"

AssertionError: Run has been registered!

In [ ]:
mlflow.register_model(model_uri = MODEL_URI, name="nyctaxi_tripduration_regressor")

In [30]:
model_name = "NYC-Taxi-Regressor"
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_52894/3905537006.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [31]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1728058774867, current_stage='Staging', description='CatBoost', last_updated_timestamp=1728059189899, name='NYC-Taxi-Regressor', run_id='1e40c7e0866c4012901bbc81fd564e8c', run_link='', source='/workspaces/MLOps/01-intro/mlruns/3/1e40c7e0866c4012901bbc81fd564e8c/artifacts/models_mlflow', status='READY', status_message='', tags={}, user_id='', version='1'>,
 <ModelVersion: aliases=[], creation_timestamp=1728058797615, current_stage='Production', description='XGBoost', last_updated_timestamp=1728059178331, name='NYC-Taxi-Regressor', run_id='324ff1de07134881b72dd65c29344d84', run_link='', source='/workspaces/MLOps/01-intro/mlruns/3/324ff1de07134881b72dd65c29344d84/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>]

In [33]:
for version in latest_versions:
        print(f"Version: {version.version}, Stage: {version.current_stage}, Model: {version.description}")

Version: 1, Stage: Staging, Model: CatBoost
Version: 2, Stage: Production, Model: XGBoost


In [ ]:
stage = "Staging"

version = 2

client.transition_model_version_stage(
    name = model_name,
    version = version,
    stage = stage,
    archive_existing_versions = False
)

In [ ]:
mlflow.register_model(model_uri = MODEL_URI, name="nyctaxi_tripduration_regressor")

Successfully registered model 'nyctaxi_tripduration_regressor'.
2024/10/05 09:07:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyctaxi_tripduration_regressor, version 1
Created version '1' of model 'nyctaxi_tripduration_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1728119220068, current_stage='None', description='', last_updated_timestamp=1728119220068, name='nyctaxi_tripduration_regressor', run_id='730586bbc7324f41bfd66cfd2b069517', run_link='', source='gs://mlflow-bucket-susu/1/730586bbc7324f41bfd66cfd2b069517/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [ ]:
RUN_ID = "b6ac39fd37a54b11a8394ea99caf5e14"
MODEL_URI = f"runs:/{RUN_ID}/model"
assert client.search_model_versions(f"run_id = '{RUN_ID}'") == [], "Run has been registered!"
mlflow.register_model(model_uri = MODEL_URI, name="nyctaxi_tripduration_regressor")

Registered model 'nyctaxi_tripduration_regressor' already exists. Creating a new version of this model...
2024/10/05 09:10:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyctaxi_tripduration_regressor, version 2
Created version '2' of model 'nyctaxi_tripduration_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1728119401861, current_stage='None', description='', last_updated_timestamp=1728119401861, name='nyctaxi_tripduration_regressor', run_id='b6ac39fd37a54b11a8394ea99caf5e14', run_link='', source='gs://mlflow-bucket-susu/1/b6ac39fd37a54b11a8394ea99caf5e14/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [ ]:
model_name = "nyctaxi_tripduration_regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


In [ ]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1728119401861, current_stage='Staging', description='', last_updated_timestamp=1728119653928, name='nyctaxi_tripduration_regressor', run_id='b6ac39fd37a54b11a8394ea99caf5e14', run_link='', source='gs://mlflow-bucket-susu/1/b6ac39fd37a54b11a8394ea99caf5e14/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [ ]:
model_name = "nyctaxi_tripduration_regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None
version: 2, stage: Staging


In [ ]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1728119220068, current_stage='Production', description='', last_updated_timestamp=1728119702204, name='nyctaxi_tripduration_regressor', run_id='730586bbc7324f41bfd66cfd2b069517', run_link='', source='gs://mlflow-bucket-susu/1/730586bbc7324f41bfd66cfd2b069517/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [ ]:
model_name = "nyctaxi_tripduration_regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging


In [ ]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": RMSE(y_test, y_pred)}

In [ ]:
model_name = "nyctaxi_tripduration_regressor"
stage = "Production"

%time test_model(name=model_name, stage=stage, X_test=X_test, y_test=y_test)


CPU times: user 613 ms, sys: 27.6 ms, total: 640 ms
Wall time: 4.15 s


{'rmse': 18.118447072717935}

In [ ]:
model_name = "nyctaxi_tripduration_regressor"
stage = "Staging"

%time test_model(name=model_name, stage=stage, X_test=X_test, y_test=y_test)


CPU times: user 643 ms, sys: 35.7 ms, total: 678 ms
Wall time: 5.4 s


{'rmse': 18.353562756759064}